In [1]:
import graphlab as gl

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.4.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
import numpy as np
import scipy as sp
import pandas as pd

# 1.Reading Data

In [3]:
features = gl.SFrame.read_csv("features.csv")

2016-03-25 19:06:05,288 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.8.4 started. Logging: C:\Users\RAHMAT~1\AppData\Local\Temp\graphlab_server_1458921963.log.0


Finished parsing file C:\Users\Rahmatulin\Documents\Coursera\HSE\kaggle\features.csv

Parsing completed. Parsed 100 lines in 0.67402 secs.

This non-commercial license of GraphLab Create is assigned to serzzh@gmail.com and will expire on October 11, 2016. For commercial licensing options, visit https://dato.com/buy/.
------------------------------------------------------

Finished parsing file C:\Users\Rahmatulin\Documents\Coursera\HSE\kaggle\features.csv

Parsing completed. Parsed 97230 lines in 0.898019 secs.


Inferred types from first line of file as 
column_type_hints=[long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,float,float,float,str,float,float,float,long,long,long,long,float,float,float,float,long,long,long,long,float,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [4]:
features_check = gl.SFrame.read_csv("features_test.csv")

Finished parsing file C:\Users\Rahmatulin\Documents\Coursera\HSE\kaggle\features_test.csv

Parsing completed. Parsed 100 lines in 0.122004 secs.

Finished parsing file C:\Users\Rahmatulin\Documents\Coursera\HSE\kaggle\features_test.csv

Parsing completed. Parsed 17177 lines in 0.143 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,long,float,float,float,str,float,float,float,long,long,long,long,float,float,float,float,long,long,long,long,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [5]:
#features_check.print_rows(num_rows=3, num_columns=109)

# 2. Removing irrelevant columns

In [6]:
name_vector = features_check.column_names()[2:]

# 3. Reshaping data

In [7]:
for f in name_vector:
    features = features.fillna(f,0)

In [8]:
data = features.to_dataframe()
y = data['radiant_win']

In [9]:
num_heroes = len(data['r1_hero'].unique()) 
r_new = []
for i in xrange(num_heroes):
    r_new += ['r'+str(i+1)+'hero','r'+str(i+1)+'level', 'r'+str(i+1)+'xp', 'r'+str(i+1)+'gold', 'r'+str(i+1)+'lh',
             'r'+str(i+1)+'kills','r'+str(i+1)+'deaths','r'+str(i+1)+'items', 'd'+str(i+1)+'hero', 'd'+str(i+1)+'level', 
             'd'+str(i+1)+'xp', 'd'+str(i+1)+'gold', 'd'+str(i+1)+'lh','d'+str(i+1)+'kills','d'+str(i+1)+'deaths',
             'd'+str(i+1)+'items']
 

In [10]:
data=data.fillna(0)
r1 = data['r1_hero']*16-16
r2 = data['r2_hero']*16-16
r3 = data['r3_hero']*16-16
r4 = data['r4_hero']*16-16
r5 = data['r5_hero']*16-16
d1 = data['d1_hero']*16-16
d2 = data['d2_hero']*16-16
d3 = data['d3_hero']*16-16
d4 = data['d4_hero']*16-16
d5 = data['d5_hero']*16-16
r = np.transpose([r1,r2,r3,r4,r5,d1,d2,d3,d4,d5])
names = list(data.columns.values)
row = []
col = []
val = []
for i in xrange(len(data)):
    for k in xrange(5):
        for j in xrange(8):
            row.insert(j+8*k+i*1000,i)
            col.insert(j+8*k+i*1000,r[i,k]+j)
            val.insert(j+8*k+i*1000,data[names[3+j+8*k]][i])

In [11]:
nele=len(data)
nbus=np.amax(r)+8
sp_features = sp.sparse.coo_matrix((val, (row,col)), shape=(nele, nbus), dtype=float)

In [12]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

def load_sparse_csr(filename):
    loader = np.load(filename)
    return sp.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [13]:
sp_csr = sp_features.tocsr()
save_sparse_csr('X_sparse',sp_csr)

In [14]:
#sp_csr = load_sparse_csr('X_sparse.npz')

In [15]:
data_sub=data[name_vector]
data_sub=data_sub[name_vector[81:]]
data_sub['first_blood_player2']=data_sub['first_blood_player2'].convert_objects(convert_numeric=True)

In [16]:
data_sub=data_sub.fillna(0)

In [17]:
names1 = list(data_sub.columns.values)
row1 = row
col1 = col
val1 = val
for i in xrange(len(data_sub.columns)):
    val1.extend(data_sub[names1[i]])
    row1.extend(range(len(data_sub)))
    col1.extend(i*np.ones(len(data_sub)))

In [18]:
nele1=len(data)
nbus1=len(data_sub.columns)+sp_csr.shape[1]
X_add = sp.sparse.coo_matrix((val1, (row1,col1)), shape=(nele1, nbus1), dtype=float)
X_csr = X_add.tocsr()

In [19]:
X = X_csr

# 3. Splitting train and test

In [20]:
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import sklearn.metrics

In [21]:
X_train, X_test, y_train, y_test = train_test_split(
...     X, y, test_size=0.1, random_state=42)

# 5. Logistics regression

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
model1 = LogisticRegression(penalty='l2')

In [24]:
model1.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [83]:
y_pred1 = model1.predict(X_test)

In [84]:
accuracy = sklearn.metrics.accuracy_score(y_test,y_pred1) 
precision = sklearn.metrics.precision_score(y_test,y_pred1) 
recall = sklearn.metrics.recall_score(y_test,y_pred1) 
f1 = sklearn.metrics.f1_score(y_test,y_pred1) 
print accuracy, precision, recall, f1

0.647639617402 0.655106342211 0.677700693756 0.666212003118


In [85]:
y_pred1_train = model1.predict(X_train)

# 6. KNeighbours

In [86]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

In [87]:
clf = KNeighborsClassifier(n_neighbors=5);

In [88]:
clf.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=5, p=2, weights='uniform')

In [90]:
max_acc=0
max_i=0
for i in range(5,6): 
    clf = KNeighborsClassifier(n_neighbors=i);
    acc=cross_val_score(clf, X_train, y_train, cv=4, scoring='accuracy').mean();
    if max_acc<acc:
        max_acc=acc;
        max_i=i
print max_acc
print max_i

MemoryError: 

# Graphlab: split train test

In [275]:
features_train, features_test = features.random_split(.9, seed=5)

gy_train = features_train['radiant_win']

gy_test = features_test['radiant_win']

GX_test = features_test[name_vector]

GX_train = features_train[name_vector]

# 4. Analysing and filling NAs

In [276]:
na_names = list()
for f in name_vector:
    N = sum(GX_train[f] == None)
    if N > 0:
        na_names.extend([f])
        print f," contains NA:", sum(GX_train[f] == None)

first_blood_time  contains NA: 17589
first_blood_team  contains NA: 17589
first_blood_player1  contains NA: 17589
radiant_bottle_time  contains NA: 14141
radiant_courier_time  contains NA: 618
radiant_flying_courier_time  contains NA: 24696
radiant_first_ward_time  contains NA: 1625
dire_bottle_time  contains NA: 14502
dire_courier_time  contains NA: 610
dire_flying_courier_time  contains NA: 23425
dire_first_ward_time  contains NA: 1634


In [280]:
for f in na_names:
    print f
    GX_train = GX_train.fillna(f, 0)
    GX_test = GX_test.fillna(f, 0)

first_blood_time
first_blood_team
first_blood_player1
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


In [281]:
GX_test['target'] = gy_test
GX_train['target'] = gy_train

In [282]:
model_g1 = gl.logistic_classifier.create(GX_train, target='target', l2_penalty = 0.0001)

Logistic regression:

--------------------------------------------------------

Number of examples          : 83062

Number of classes           : 2

Number of feature columns   : 101

...

| 3         | 4        | 22.458160    | 0.656473          | 0.652303            |

| 4         | 5        | 27.384298    | 0.656473          | 0.652303            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



In [283]:
gy_pred1 = model_g1.predict(GX_test)

In [287]:
gl.evaluation.roc_curve(gy_test, gy_pred1).tail()

threshold,fpr,tpr,p,n
0.99991,0.393484239475,0.696538081108,5055,4727
0.99992,0.393484239475,0.696538081108,5055,4727
0.99993,0.393484239475,0.696538081108,5055,4727
0.99994,0.393484239475,0.696538081108,5055,4727
0.99995,0.393484239475,0.696538081108,5055,4727
0.99996,0.393484239475,0.696538081108,5055,4727
0.99997,0.393484239475,0.696538081108,5055,4727
0.99998,0.393484239475,0.696538081108,5055,4727
0.99999,0.393484239475,0.696538081108,5055,4727
1.0,0.0,0.0,5055,4727


In [288]:
gl.evaluation.accuracy(gy_test, gy_pred1)

0.6530361889184216

In [285]:
gy_pred1_train = model_g1.predict(GX_train)